# Example notebook

In [1]:
using Random, Statistics, CSV, DataFrames
using LinearAlgebra

In [2]:
include("../impute.jl")
include("../regress.jl")
include("../augment.jl")
include("../count.jl")

size

In [3]:
dataset_list = [d for d in split.(read(`ls ../datasets/`, String), "\n") if length(d) > 0]

32-element Array{SubString{String},1}:
 "arrhythmia"                        
 "auto-mpg"                          
 "automobile"                        
 "breast-cancer-wisconsin-original"  
 "breast-cancer-wisconsin-prognostic"
 "communities-and-crime"             
 "communities-and-crime-2"           
 "credit-approval"                   
 "cylinder-bands"                    
 "dermatology"                       
 "echocardiogram"                    
 "heart-disease-cleveland"           
 "heart-disease-hungarian"           
 ⋮                                   
 "ozone-level-detection-one"         
 "sleep"                             
 "thyroid-disease-allbp"             
 "thyroid-disease-allhyper"          
 "thyroid-disease-allhypo"           
 "thyroid-disease-allrep"            
 "thyroid-disease-dis"               
 "thyroid-disease-sick"              
 "thyroid-disease-sick-euthyroid"    
 "thyroid-disease-thyroid-0387"      
 "trains"                            
 "wiki4he" 

In [4]:
results_table = DataFrame(dataset=[], copynum=[], iter=[], method=[], osr2=[])

,dataset,copynum,iter,method,osr2
,Any,Any,Any,Any,Any


In [5]:
SNR = 4

4

In [ ]:
if !isdir("../results")
    mkdir("../results")
end

In [22]:
dname = "sleep" #"""thyroid-disease-thyroid-0387" #dataset_list[1]
i = 1
iter = 1

1

In [57]:
?CSV.File

```
CSV.File(source; kwargs...) => CSV.File
```

Read a csv input (a filename given as a String or FilePaths.jl type, or any other IO source), returning a `CSV.File` object.

Opens the file and uses passed arguments to detect the number of columns and column types. The returned `CSV.File` object supports the [Tables.jl](https://github.com/JuliaData/Tables.jl) interface and can iterate `CSV.Row`s. `CSV.Row` supports `propertynames` and `getproperty` to access individual row values. Note that duplicate column names will be detected and adjusted to ensure uniqueness (duplicate column name `a` will become `a_1`). For example, one could iterate over a csv file with column names `a`, `b`, and `c` by doing:

```julia
for row in CSV.File(file)
    println("a=$(row.a), b=$(row.b), c=$(row.c)")
end
```

By supporting the Tables.jl interface, a `CSV.File` can also be a table input to any other table sink function. Like:

```julia
# materialize a csv file as a DataFrame
df = CSV.File(file) |> DataFrame!

# load a csv file directly into an sqlite database table
db = SQLite.DB()
tbl = CSV.File(file) |> SQLite.load!(db, "sqlite_table")
```

Supported keyword arguments include:

  * File layout options:

      * `header=1`: the `header` argument can be an `Int`, indicating the row to parse for column names; or a `Range`, indicating a span of rows to be concatenated together as column names; or an entire `Vector{Symbol}` or `Vector{String}` to use as column names; if a file doesn't have column names, either provide them as a `Vector`, or set `header=0` or `header=false` and column names will be auto-generated (`Column1`, `Column2`, etc.)
      * `normalizenames=false`: whether column names should be "normalized" into valid Julia identifier symbols; useful when iterating rows and accessing column values of a row via `getproperty` (e.g. `row.col1`)
      * `datarow`: an `Int` argument to specify the row where the data starts in the csv file; by default, the next row after the `header` row is used. If `header=0`, then the 1st row is assumed to be the start of data
      * `skipto::Int`: similar to `datarow`, specifies the number of rows to skip before starting to read data
      * `footerskip::Int`: number of rows at the end of a file to skip parsing
      * `limit`: an `Int` to indicate a limited number of rows to parse in a csv file; use in combination with `skipto` to read a specific, contiguous chunk within a file
      * `transpose::Bool`: read a csv file "transposed", i.e. each column is parsed as a row
      * `comment`: rows that begin with this `String` will be skipped while parsing
      * `use_mmap::Bool=!Sys.iswindows()`: whether the file should be mmapped for reading, which in some cases can be faster
      * `ignoreemptylines::Bool=false`: whether empty rows/lines in a file should be ignored (if `false`, each column will be assigned `missing` for that empty row)
      * `threaded::Bool`: whether parsing should utilize multiple threads; by default threads are used on large enough files, but isn't allowed when `transpose=true` or when `limit` is used; only available in Julia 1.3+
  * Parsing options:

      * `missingstrings`, `missingstring`: either a `String`, or `Vector{String}` to use as sentinel values that will be parsed as `missing`; by default, only an empty field (two consecutive delimiters) is considered `missing`
      * `delim=','`: a `Char` or `String` that indicates how columns are delimited in a file; if no argument is provided, parsing will try to detect the most consistent delimiter on the first 10 rows of the file
      * `ignorerepeated::Bool=false`: whether repeated (consecutive) delimiters should be ignored while parsing; useful for fixed-width files with delimiter padding between cells
      * `quotechar='"'`, `openquotechar`, `closequotechar`: a `Char` (or different start and end characters) that indicate a quoted field which may contain textual delimiters or newline characters
      * `escapechar='"'`: the `Char` used to escape quote characters in a quoted field
      * `dateformat::Union{String, Dates.DateFormat, Nothing}`: a date format string to indicate how Date/DateTime columns are formatted for the entire file
      * `decimal='.'`: a `Char` indicating how decimals are separated in floats, i.e. `3.14` used '.', or `3,14` uses a comma ','
      * `truestrings`, `falsestrings`: `Vectors of Strings` that indicate how `true` or `false` values are represented; by default only `true` and `false` are treated as `Bool`
  * Column Type Options:

      * `type`: a single type to use for parsing an entire file; i.e. all columns will be treated as the same type; useful for matrix-like data files
      * `types`: a Vector or Dict of types to be used for column types; a Dict can map column index `Int`, or name `Symbol` or `String` to type for a column, i.e. Dict(1=>Float64) will set the first column as a Float64, Dict(:column1=>Float64) will set the column named column1 to Float64 and, Dict("column1"=>Float64) will set the column1 to Float64; if a `Vector` if provided, it must match the # of columns provided or detected in `header`
      * `typemap::Dict{Type, Type}`: a mapping of a type that should be replaced in every instance with another type, i.e. `Dict(Float64=>String)` would change every detected `Float64` column to be parsed as `String`
      * `pool::Union{Bool, Float64}=0.1`: if `true`, *all* columns detected as `String` will be internally pooled; alternatively, the proportion of unique values below which `String` columns should be pooled (by default 0.1, meaning that if the # of unique strings in a column is under 10%, it will be pooled)
      * `categorical::Bool=false`: whether pooled columns should be copied as CategoricalArray instead of PooledArray; note that in `CSV.read`, by default, columns are not copied, so pooled columns will have type `CSV.Column{String, PooledString}`; to get `CategoricalArray` columns, also pass `copycols=true`
      * `strict::Bool=false`: whether invalid values should throw a parsing error or be replaced with `missing`
      * `silencewarnings::Bool=false`: if `strict=false`, whether invalid value warnings should be silenced


Read in a data file.

In [58]:
X_missing = DataFrame(CSV.read("../datasets/"*dname*"/$i/X_missing.csv", missingstrings=["", "NaN"]))
X_full = DataFrame(CSV.read("../datasets/"*dname*"/$i/X_full.csv"))
first(X_full, 5)

,BodyWgt,BrainWgt,NonD,Dream,Sleep,Span,Gest,Pred
,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,7.17917,5.83574,-1.02904,-0.0499081,-1.57003,1.02835,3.42391,0.0873957
2,-0.219972,-0.297259,-0.647197,0.0194087,-0.48467,-0.844632,-0.683583,0.0873957
3,-0.21732,-0.256519,0.634696,0.573944,0.427034,-0.322833,-0.560972,-1.26724
4,-0.220061,-0.298227,1.39838,0.43531,1.29532,-0.817169,-0.799383,1.44203
5,2.61157,4.64362,-1.79272,-0.119225,-1.43979,2.69811,3.28086,0.0873957


Create output

In [59]:
function softthresholding(x; λ=0.1)
    if x > λ
        return x - λ
    elseif x < -λ
        return x + λ
    else 
        return 0 
    end
end

softthresholding (generic function with 1 method)

In [60]:
Base.size(X_full)

(62, 11)

In [61]:
n,p = Base.size(X_full)
wtrue = softthresholding.(randn(p))
btrue = rand(1)

test_index = findfirst(names(X_full) .== :Test)
wtrue[test_index] = 0.

μ = mean(Matrix{Float64}(X_full), dims=1)
σ = std(Matrix{Float64}(X_full), dims=1)
σ[findall(σ .== 0)].= 1
X_normalize = (X_full .- μ) ./ σ

Y = Matrix{Float64}(X_normalize)*wtrue .+ btrue

62-element Array{Float64,1}:
 -6.4985273940713055 
  2.5876958463564903 
  0.6017918966004122 
  2.140264322024931  
 -8.422035430389734  
  0.6743387931242566 
 -0.18967584305253915
 -2.151882287267794  
 -0.8386794465391496 
 -2.6778861841866073 
  2.03167876614024   
 -0.20032540862374137
  2.1214350759641762 
  ⋮                  
  0.4428244276862881 
  1.1793261241829565 
  2.4047454085655904 
  1.8767376363827495 
  0.8622389851456416 
  2.157053761157892  
  1.7046459930469406 
  1.4811759031834837 
  1.6528996125665032 
  0.6613898760704107 
 -0.7301564988727616 
  1.644800914760108  

In [27]:
noise = randn(Base.size(X_full,1))
noise .*= norm(Y)/norm(noise)/SNR
Y .+= noise

62-element Array{Float64,1}:
  3.243129078514413   
  1.0938642837343284  
 -0.37385981282437486 
  2.232672696705897   
  5.948635002212793   
  1.223104258098572   
  0.18739067289471578 
 -1.3000519534990858  
  0.14558106927612502 
  2.5427076361485677  
  1.4625406770401659  
  2.0280791242731437  
  0.2155201076902763  
  ⋮                   
 -2.2622537833975205  
 -1.169358405753589   
  0.9660056518700961  
  1.6206089682068492  
  0.029848989016576816
  1.0949054509603524  
 -0.10340540831949625 
 -0.06837715832951548 
  0.512558439948992   
  2.4219155500695786  
  1.6003581165265885  
  0.7999168653099576  

Add Offset

X_full[:,:One] = Ref(1.)
X_missing[:,:One] = Ref(1.) ;

- Method 1: Impute missing data using MICE and regress using LASSO.

In [62]:
X_imputed = mice(X_missing);
first(X_imputed, 5)

,BodyWgt,BrainWgt,NonD,Dream,Sleep,Span,Gest,Pred
,Float64,Float64,Float64⍰,Float64⍰,Float64⍰,Float64⍰,Float64⍰,Float64
1,7.17917,5.83574,-1.79272,-0.535126,-1.57003,1.02835,3.42391,0.0873957
2,-0.219972,-0.297259,-0.647197,0.0194087,-0.48467,-0.844632,-0.683583,0.0873957
3,-0.21732,-0.256519,0.634696,-0.396492,0.427034,-0.322833,-0.560972,-1.26724
4,-0.220061,-0.298227,1.12563,0.989845,1.29532,-0.73478,-0.799383,1.44203
5,2.61157,4.64362,-1.79272,-0.119225,-1.43979,2.69811,3.28086,0.0873957


In [63]:
linear = regress(Y, X_imputed, lasso=true, alpha=0.8)

,BodyWgt,BrainWgt,NonD,Dream,Sleep,Span,Gest,Pred
,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,0.0,-0.266207,0.0,-0.676226,-0.132777,-0.973766,-1.71806,0.0


In [64]:
R2, OSR2 = evaluate(Y, X_imputed, linear)
@show R2
@show OSR2

R2 = 0.9870496052410258
OSR2 = 0.9752480162513236


0.9752480162513236

In [65]:
push!(results_table, [dname, i, iter, "Impute then regress", OSR2])

,dataset,copynum,iter,method,osr2
,Any,Any,Any,Any,Any
1,sleep,1,1,Impute then regress,0.975248


- Method 2: Add indicator variables for missingness and impute zeros (finitely adaptive). Regress using LASSO.

In [66]:
X_augmented = hcat(zeroimpute(X_missing), indicatemissing(X_missing, removezerocols=true))
linear2 = regress(Y, X_augmented, lasso=true, alpha=0.8, missing_penalty=2.0)

,BodyWgt,BrainWgt,NonD,Dream,Sleep,Span,Gest,Pred
,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,0.0,-0.159848,0.23717,-0.754339,-0.405644,-0.98163,-1.84038,0.0


In [68]:
X_augmented

,BodyWgt,BrainWgt,NonD,Dream,Sleep,Span,Gest
,Float64,Float64,Float64⍰,Float64⍰,Float64⍰,Float64⍰,Float64⍰
1,7.17917,5.83574,0.0,0.0,-1.57003,1.02835,3.42391
2,-0.219972,-0.297259,-0.647197,0.0194087,-0.48467,-0.844632,-0.683583
3,-0.21732,-0.256519,0.0,0.0,0.427034,-0.322833,-0.560972
4,-0.220061,-0.298227,0.0,0.0,1.29532,0.0,-0.799383
5,2.61157,4.64362,-1.79272,-0.119225,-1.43979,2.69811,3.28086
6,-0.209351,-0.111401,0.116484,-0.88171,-0.159062,0.391207,0.256439
7,-0.221059,-0.304032,1.94386,1.33643,1.98995,-0.0482025,-0.731265
8,-0.0431403,-0.122688,-0.947215,-0.67376,-0.940522,0.577956,1.70053
9,-0.217414,-0.276835,0.607422,1.12848,0.861178,0.446133,-0.540536


In [53]:
indicatemissing(X_missing, removezerocols=true)

┌ Warning: `getindex(df::DataFrame, col_ind::ColumnIndex)` is deprecated, use `df[!, col_ind]` instead.
│   caller = #indicatemissing#14(::Bool, ::Function, ::DataFrame) at In[52]:5
└ @ Main ./In[52]:5


,BodyWgt_missing,BrainWgt_missing,NonD_missing,Dream_missing,Sleep_missing,Span_missing
,Int64,Int64,Int64,Int64,Int64,Int64
1,0,0,0,0,0,0
2,0,0,0,0,0,0
3,0,0,0,0,0,0
4,0,0,0,0,0,0
5,0,0,0,0,0,0
6,0,0,0,0,0,0
7,0,0,0,0,0,0
8,0,0,0,0,0,0
9,0,0,0,0,0,0


In [69]:
R2, OSR2 = evaluate(Y, X_augmented, linear2)
@show R2
@show OSR2

R2 = 0.9966494716573744
OSR2 = 0.9852525510283157


0.9852525510283157

In [ ]:
push!(results_table, [dname, i, iter, "Augmented", OSR2])

- Method 3: Affinely adaptive

In [20]:
X_affine = augmentaffine(X_missing, removezerocols=true)
linear3 = regress(Y, X_affine, lasso=true, alpha=0.8, missing_penalty=10.0)

┌ Warning: `getindex(df::DataFrame, col_ind::ColumnIndex)` is deprecated, use `df[!, col_ind]` instead.
│   caller = #augmentaffine#5(::Bool, ::Function, ::DataFrame) at augment.jl:45
└ @ Main /Users/jpauph/Dropbox (MIT)/1- Research/PHD/augment.jl:45
┌ Warning: `getindex(df::DataFrame, col_ind::ColumnIndex)` is deprecated, use `df[!, col_ind]` instead.
│   caller = #augmentaffine#5(::Bool, ::Function, ::DataFrame) at augment.jl:45
└ @ Main /Users/jpauph/Dropbox (MIT)/1- Research/PHD/augment.jl:45
┌ Warning: `setindex!(df::DataFrame, v::AbstractVector, col_ind::ColumnIndex)` is deprecated, use `begin
│     df[!, col_ind] = v
│     df
│ end` instead.
│   caller = #augmentaffine#5(::Bool, ::Function, ::DataFrame) at augment.jl:45
└ @ Main /Users/jpauph/Dropbox (MIT)/1- Research/PHD/augment.jl:45


,N,N_1,N_2,N_3,N_4,N_5,N_6,N_1_missing
,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,3.8973e-5,-0.0241811,0.495174,-0.0289264,1.56968,0.00219125,0.00235848,0.0


In [21]:
R2, OSR2 = evaluate(Y, X_affine, linear3)
@show R2
@show OSR2

R2 = 0.8084333591361579
OSR2 = 0.7989896100866425


0.7989896100866425

In [ ]:
push!(results_table, [dname, i, iter, "Augmented Affine", OSR2])

In [ ]:
CSV.write("../results/"*dname*"_$i.csv", results_table)